In [ ]:
import requests, os
import pandas as pd

# ===== 設定 =====
URL = "https://webservice.recruit.co.jp/hotpepper/gourmet/v1/"
API_KEY = "ここにご自身のAPIキーを入力してください"   # ※ 本来は環境変数推奨
KEYWORD = "五反田駅"
START = 1
COUNT = 100
MAX_BUDGET = 4000

# ===== APIリクエスト =====
params = {
    "key": API_KEY,
    "keyword": KEYWORD,
    "format": "json",
    "start": START,
    "count": COUNT
}

res = requests.get(URL, params=params)
json_data = res.json()

shops = json_data["results"]["shop"]

# ===== データ整形 =====
data = []
for shop in shops:
    data.append({
        "name": shop["name"],
        "address": shop["address"],
        "capacity": shop["capacity"],
        "access": shop["access"],
        "open": shop["open"],
        "budget": shop["budget"]["name"],
        "url": shop["urls"]["pc"]
    })

df = pd.DataFrame(data)

# ===== 予算文字列を数値に変換 =====
def parse_budget(budget):
    if not budget:
        return 0, 0
    lower, upper = budget.replace("円", "").split("～")
    return int(lower), int(upper)

df[["lower_budget", "upper_budget"]] = df["budget"].apply(
    lambda x: pd.Series(parse_budget(x))
)

# ===== 予算フィルタ =====
df_filtered = df[df["upper_budget"] <= MAX_BUDGET]

df_filtered.head()


,name,address,capacity,access,open,budget,url,lower_budget,upper_budget
1,幻の和牛と三崎港直送鮮魚 MORGEN 五反田駅前店,東京都品川区西五反田１-18-1 五反田EXビル7F,70,五反田駅徒歩3分／大崎広小路駅徒歩1分／大崎駅徒歩10分,月、日、祝日: 17:00～22:30 （料理L.O. 22:00 ドリンクL.O. 22:...,3001～4000円,https://www.hotpepper.jp/strJ001278727/?vos=nh...,3001,4000
2,海坊主五反田駅前店,東京都品川区西五反田２-７-８ 誠實ビル7階,95,ＪＲ五反田駅西口より徒歩約2分,月～日、祝日、祝前日: 11:00～14:30 （料理L.O. 14:00 ドリンクL.O....,3001～4000円,https://www.hotpepper.jp/strJ004402152/?vos=nh...,3001,4000
3,居酒屋 せんや 五反田駅本店,東京都品川区東五反田１-17-7大宗五反田ビル202,100,各線五反田駅東口徒歩２分／鹿児島黒牛や鮮度抜群の刺身を完全個室で堪能♪,月～金、日、祝前日: 16:00～翌0:00 （料理L.O. 23:00 ドリンクL.O. ...,3001～4000円,https://www.hotpepper.jp/strJ001244947/?vos=nh...,3001,4000
4,酒蔵 桜ちょうちん 五反田,東京都品川区西五反田１‐4‐2 秀和五反田駅前レジデンス2階,150,JR五反田駅徒歩２分/都営浅草線五反田駅徒歩２分/東急池上線五反田駅徒歩２分,月～金: 16:00～23:30 （料理L.O. 22:30 ドリンクL.O. 23:00）...,3001～4000円,https://www.hotpepper.jp/strJ003893833/?vos=nh...,3001,4000
5,0秒レモンサワー 仙台ホルモン焼肉酒場 ときわ亭 五反田店,東京都品川区西五反田１丁目４－８ 秀和五反田駅前レジデンス ２Ｆ,63,JR 五反田駅 徒歩2分 都営浅草線五反田駅徒歩2分★ときわ亭五反田店★,月～木: 17:00～23:30 （料理L.O. 22:30 ドリンクL.O. 22:30）...,2001～3000円,https://www.hotpepper.jp/strJ001266670/?vos=nh...,2001,3000
